In [1]:
import torch
from transformers import BertConfig, BertForQuestionAnswering

# 체크포인트 설정

In [2]:
from ratsnlp.nlpbook.qa import QADeployArguments
dargs = QADeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='.checkpoint-qa',
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: .checkpoint-qa/epoch=1-val_loss=4.85.ckpt


# 모델 로딩

In [3]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    dargs.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu'),
)

In [4]:
# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    dargs.pretrained_model_name
)
# BERT 모델 초기화
model = BertForQuestionAnswering(pretrained_model_config)

In [5]:
# 체크포인트 주입
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [6]:
# 평가 모드
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    dargs.pretrained_model_name,
    do_lower_case=False,
)

# 모델 출력 및 후처리

In [15]:
def inference_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=dargs.max_query_length
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=dargs.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [16]:
context = "한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다."
question = "한강대교 아래에는 어떤 섬이 있는가?"
inference_fn(question=question, context=context)

{'question': '한강대교 아래에는 어떤 섬이 있는가?',
 'context': '한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다.',
 'answer': '[CLS]'}